In [162]:
import numpy as np
import torch

from torch.utils.data import Dataset

import torch.nn.functional as F
import torch.nn as nn

import matplotlib.pyplot as plt

import myModelinTF

from torch.optim.lr_scheduler import StepLR
import pandas as pd
import tensorflow as tf
from torch.autograd import Variable 

In [163]:
class SeriesDataset(Dataset):

    def __init__(self, file):
        self.demo = np.load(file, allow_pickle=True)

    #len(usd)
    def __len__(self):
        return len(self.demo)

    # a_list[1] --> a__list.__getitem__(1)
    def __getitem__(self, index):
        run =self.demo[index]

        states = np.array(run[0]).T
        states_init1 = states[1,:]
        states_init = np.zeros((500, 10)) + states_init1
        actions = np.array(run[1])[:, None]
        return states_init, actions

In [164]:
def permute_them(x):
  return torch.permute(x, (1, 0, 2))

In [165]:
def collate_fn(batch):

    # A data tuple has the form:

    tensors, targets = [], []

    # Gather in lists, and encode labels as indices
    for states_init, actions in batch:
        tensors += [torch.tensor(states_init).type(torch.FloatTensor)]
        targets += [torch.tensor(actions).type(torch.LongTensor)]

    # Group the list of tensors into a batched tensor
    targets = torch.stack(targets)

    return tensors, targets

In [166]:
class PG(nn.Module):
    def __init__(self, state_shape, n_actions):
        super().__init__()
        self.state_shape = state_shape
        self.n_actions = n_actions

        # Policy takes input initial conditions and outputs planning route, open-loop control
        self.model = nn.Sequential(
            nn.LSTM(input_size = 10, hidden_size = 4, num_layers = 1, dropout = 0.2,batch_first = True)
        )
        self.optimizer = torch.optim.Adam(self.model.parameters(), 1e-3)

    def forward(self, states):
        #states = torch.FloatTensor(states)
        logits, (h_T, c_T)  = self.model(states)
        return logits
    
    def predict_probs(self, states):
        states = torch.FloatTensor(states)
        logits, (h_T, c_T) = self.model(states)
        probs = F.softmax(logits, dim = -1).detach().numpy()
        return probs
    
    # Run agent in environment to create sample trajectories by generator
    # The environment model is a seq2seq model
    def generate_session(self, sysmodel, t_max=1000):
        norm_params = pd.read_csv('D:\\units\\thesis\\thesis\\train_reward_inverse_rl_w_sys_model\\norm_params.csv', header = None).to_numpy().T

        states, actions, traj_probs, rewards = [], [], [], []
        states_init = np.zeros((500,1, 10)) + np.random.randint(20, 34, size=(10)) 
        actions_probs_policy = self.predict_probs(states_init)
        actions = []
        actions_probs_policy = np.squeeze(actions_probs_policy)     
        for prob in actions_probs_policy:                  
            actions.append(np.random.choice(self.n_actions,  p = prob))
 
        # Apply dict to go from 1,2,3,4 as action to [0,2400,0] [0,2400,1000]  [2300,0,0]  [2300,0,1000] 
        my_dict = {0:[0,2400,0], 1:[0,2400,1000], 2:[2300,0,0], 3:[2300,0,1000]}
        actions_array = np.zeros((500,3))  
        actions_array = map(my_dict.get, actions)

        actions_array = np.array(list(actions_array))
        actions_array_sysmodel = np.concatenate([np.arange(actions_array.shape[0])[:,None]+1,actions_array], axis=1) 
        actions_array_sysmodel = tf.expand_dims(actions_array_sysmodel, axis = 0, name=None)

        states = sysmodel(actions_array_sysmodel)
        states = states*norm_params[:,0]+norm_params[:,1]
        
        return states, actions, actions_probs_policy

In [167]:
def train(model, tepoch, log_interval):

    model.train()
    losses = []
    batch_idx = 0
    
    for data, target in train_loader: #tqdm_notebook(train_loader, total = len(train_loader)):
        data = torch.stack(data)
        
        output = model(data)
        target = to_one_hot(target, 4)
        output = torch.flatten(output, start_dim = 0, end_dim = 1)
        
        loss = criterion(output, target)

        output_np = output.detach().numpy().tolist()
        target_np = target.detach().numpy().tolist()
        correct = 0
        for (output, target) in zip(output_np, target_np):           
            if output.index(max(output)) == target.index(max(target)):
                correct +=1
        accuracy = correct / 500 * BATCH_SIZE

        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        # print training stats
        #if batch_idx % log_interval == 0:
        print(f"Train Epoch: {epoch} [{batch_idx * len(data)}/{len(train_loader.dataset)} ({100. * batch_idx / len(train_loader):.0f}%)]\tLoss: {loss.item():.6f}\tAccuracy of batch: {accuracy:.6f}")
        # record loss
        
        losses.append(loss.item())

        batch_idx += 1

In [168]:
def test(model, epoch):
    model.eval()
    correct = 0

    for data, target in test_loader:
        data = torch.stack(data)

        output = model(data)
        target = to_one_hot(target, 4)
        output = torch.flatten(output, start_dim = 0, end_dim = 1)
        
        output_np = output.detach().numpy().tolist()
        target_np = target.detach().numpy().tolist()
        correct = 0
        for (output, target) in zip(output_np, target_np):           
            if output.index(max(output)) == target.index(max(target)):
                correct +=1

        accuracy = correct / 500 * BATCH_SIZE

    print(f"\nTest Epoch: {epoch}\tAccuracy: {accuracy}/{len(test_loader.dataset)} ({100. * correct / len(test_loader.dataset):.0f}%)\n")

In [169]:
def number_of_correct(pred, target):
    # count number of correct predictions
    return pred.squeeze().eq(target).sum().item()

def get_likely_index(tensor):
    # find most likely label index for each element in the batch
    return tensor.argmax(dim=-1)

def compute_error(x, y):
    Ps = torch.norm(x)
    Pn = torch.norm(x-y)
    return 20*torch.log10(Ps/Pn)

def get_expected_rewards(rewards,gamma=0.9):
    G = np.zeros_like(rewards, dtype=float)
    G[-1] = rewards[-1]
    for idx in range(-2, -len(rewards)-1, -1):
        G[idx] = rewards[idx] + gamma * G[idx+1]
    return G

def get_rewards(states, actions):
    rewards = np.zeros((500,1))

    states = states.numpy()

    max_hotspot =  np.max(states)
    if max_hotspot < 250:
        rewards[-1] = 50

    if np.where(states[:,1] == 230)[0] < 405 & np.where(states[:,1] == 230)[0] > 340:
        rewards[-1] = rewards[-1] + 40
        
    for idx in range(len(states)):
        temps = np.array(states[idx])
        if temps.any() > 250:
            rewards[idx] = rewards[idx] -20
        
        subset = actions[idx-10:]
        if np.sum(np.diff(subset)) == 0:
            rewards[idx] = rewards[idx] + 20

        if actions[idx] == 1 | actions[idx] == 3:
            rewards[idx] = rewards[idx] -10


    return rewards

def to_one_hot(y_tensor, ndims):
    y_tensor = y_tensor.type(torch.LongTensor).view(-1, 1)
    y_one_hot = torch.zeros(
        y_tensor.size()[0], ndims).scatter_(1, y_tensor, 1)
    return y_one_hot

In [170]:
# CONVERTS TRAJ LIST TO STEP LIST
def preprocess_traj(traj_list, step_list):
    step_list = step_list.tolist()
    for traj in traj_list:
        states = np.array(traj[0])
        probs = np.ones((500,4))
        states = np.transpose(traj[0])
        actions = np.array(traj[1])[:, None]

        x = np.concatenate((states, actions, probs), axis=1) 
        step_list.extend(x)
    return np.array(step_list)


In [171]:
# DEVICE
DEVICE = 'cpu'

# ENV SETUP
sysmodel = myModelinTF.load_model()
n_actions = 4
state = np.zeros((500,10)) + 25
state_shape = state.shape

# INITILIZING POLICY AND REWARD FUNCTION
policy = PG(state_shape, n_actions)
policy.to(DEVICE)
optimizer = torch.optim.Adam(policy.parameters(), 1e-2, weight_decay=1e-4)

mean_rewards = []
mean_loss = []
mean_loss_rew = []
EPISODES_TO_PLAY = 5000
sample_trajs = []

return_list, sum_of_loss_list = [], []
norm_params = pd.read_csv('D:\\units\\thesis\\thesis\\train_reward_inverse_rl_w_sys_model\\norm_params.csv', header = None).to_numpy().T

# LOADING EXPERT/DEMO SAMPLES

# PRE-TRAIN WITH EXISTING DATA THE POLICY

LOG_INTERVAL = 2
N_EPOCH = 200

scheduler = torch.optim.lr_scheduler.StepLR(policy_optimizer, step_size=20, gamma=0.1)  # reduce the learning after 20 epochs by a factor of 10
criterion = nn.CrossEntropyLoss()

DATASETNPY_TRAIN = "dataset_train.npy"
DATASETNPY_TEST = "dataset_test.npy"

train_set = SeriesDataset(DATASETNPY_TRAIN)
test_set = SeriesDataset(DATASETNPY_TEST)

print("Train set size: " + str(len(train_set)))
BATCH_SIZE = 32

train_loader = torch.utils.data.DataLoader(
        train_set,
        batch_size=BATCH_SIZE,
        shuffle=True,
        collate_fn=collate_fn,
    )
test_loader = torch.utils.data.DataLoader(
        test_set,
        batch_size=BATCH_SIZE,
        shuffle=False,
        collate_fn=collate_fn,
    )

for epoch in range(1, N_EPOCH + 1):
    train(policy, epoch, LOG_INTERVAL)
    test(policy, epoch)
    scheduler.step()


for i in range(EPISODES_TO_PLAY):

    states_init = np.zeros((1,500, 10)) + np.random.randint(20, 34, size=(10))
    states_init = torch.FloatTensor(states_init)
    logits = policy(states_init) # forward pass
    probs = nn.functional.softmax(logits, -1) # get estimated actions for states
    #log_probs = nn.functional.log_softmax(logits, -1)

    actions = []
    actions_probs_policy = np.squeeze(probs).detach().numpy()     
    for prob in actions_probs_policy:                  
        actions.append(np.random.choice(4,  p = prob))

    # Apply dict to go from 1,2,3,4 as action to [0,2400,0] [0,2400,1000]  [2300,0,0]  [2300,0,1000] 
    my_dict = {0:[0,2400,0], 1:[0,2400,1000], 2:[2300,0,0], 3:[2300,0,1000]}
    actions_array = np.zeros((500,3))  
    actions_array = map(my_dict.get, actions)

    actions_array = np.array(list(actions_array))
    actions_array_sysmodel = np.concatenate([np.arange(actions_array.shape[0])[:,None]+1,actions_array], axis=1) 
    actions_array_sysmodel = tf.expand_dims(actions_array_sysmodel, axis = 0, name=None)

    states = sysmodel(actions_array_sysmodel)

    states = states*norm_params[:,0]+norm_params[:,1]

    # Recursively get expected discounted rewards (rewards given by the current cost function)
    rewards = get_rewards(states, actions)
    cumulative_returns_np = np.array(get_expected_rewards(rewards, 0.9))
    cumulative_returns = torch.tensor(cumulative_returns_np, dtype=torch.float32)

    actions_tensor = torch.from_numpy(np.array(actions))

    log_probs_for_actions = torch.sum(
        probs * to_one_hot(actions_tensor, 4), dim=1)

    #entropy = -torch.mean(torch.sum(probs_samp*log_probs_samp), dim = -1 )
    #loss = -torch.mean(log_probs_for_actions*cumulative_returns - entropy*1e-2) # loss for the policy (isnt it the cost function output?)
    #average reward baseline
    cumulative_returns = (cumulative_returns - torch.mean(cumulative_returns))
    #print(cumulative_returns)
    #whitening baseline
    #cumulative_returns = (cumulative_returns - torch.mean(cumulative_returns))/ (torch.std(cumulative_returns))
    loss_policy = -log_probs_for_actions*cumulative_returns
    # UPDATING THE POLICY NETWORK
    #policy_optimizer.zero_grad()
    print(loss_policy.sum().detach().numpy())
    loss_policy.sum().backward()
    torch.nn.utils.clip_grad_norm_(policy.parameters(), 0.01)
    optimizer.step()

    #returns = sum(rewards)
    sum_of_loss = loss_policy.sum().detach().numpy()
    #return_list.append(returns)
    sum_of_loss_list.append(sum_of_loss)
    states_plot = np.squeeze(states)

    plt.subplot(2, 2, 1)
    plt.title(f"Mean loss per {EPISODES_TO_PLAY} episodes")
    plt.plot(mean_loss)
    plt.grid()

    plt.subplot(2, 2, 2)
    plt.title(f"Rewards - last series")
    plt.plot(rewards)
    plt.grid()

    plt.subplot(2, 2, 3)
    plt.title(f"Preheat phase Activations - last series")
    plt.plot(actions[450:500])
    plt.grid()

    plt.subplot(2, 2, 4)
    plt.title(f"Preheat phase Temperatures - last series")
    plt.plot(states_plot[0:50,:])
    plt.grid()

    # plt.show()
    plt.savefig('GCL_learning_curve.png')
    plt.close()

d:\units\thesis\thesis\train_reward_inverse_rl_w_sys_model\.venv\lib\site-packages\torch\nn\modules\rnn.py:82: UserWarning: dropout option adds dropout after all but last recurrent layer, so non-zero dropout expects num_layers greater than 1, but got dropout=0.2 and num_layers=1
  warnings.warn("dropout option adds dropout after all but last "


Train set size: 495
Train Epoch: 1 [0/495 (0%)]	Loss: 1.923454	Accuracy of batch: 287.552000
Train Epoch: 1 [32/495 (6%)]	Loss: 1.692882	Accuracy of batch: 320.832000
Train Epoch: 1 [64/495 (12%)]	Loss: 1.536738	Accuracy of batch: 289.664000
Train Epoch: 1 [96/495 (19%)]	Loss: 1.367030	Accuracy of batch: 319.232000
Train Epoch: 1 [128/495 (25%)]	Loss: 1.242898	Accuracy of batch: 297.280000
Train Epoch: 1 [160/495 (31%)]	Loss: 1.217640	Accuracy of batch: 293.248000
Train Epoch: 1 [192/495 (38%)]	Loss: 1.216680	Accuracy of batch: 296.256000
Train Epoch: 1 [224/495 (44%)]	Loss: 1.218072	Accuracy of batch: 300.224000
Train Epoch: 1 [256/495 (50%)]	Loss: 1.216443	Accuracy of batch: 300.672000
Train Epoch: 1 [288/495 (56%)]	Loss: 1.218071	Accuracy of batch: 305.728000
Train Epoch: 1 [320/495 (62%)]	Loss: 1.218071	Accuracy of batch: 315.456000
Train Epoch: 1 [352/495 (69%)]	Loss: 1.219000	Accuracy of batch: 311.104000
Train Epoch: 1 [384/495 (75%)]	Loss: 1.216908	Accuracy of batch: 308.736000

d:\units\thesis\thesis\train_reward_inverse_rl_w_sys_model\.venv\lib\site-packages\torch\optim\lr_scheduler.py:136: UserWarning: Detected call of `lr_scheduler.step()` before `optimizer.step()`. In PyTorch 1.1.0 and later, you should call them in the opposite order: `optimizer.step()` before `lr_scheduler.step()`.  Failure to do this will result in PyTorch skipping the first value of the learning rate schedule. See more details at https://pytorch.org/docs/stable/optim.html#how-to-adjust-learning-rate
  warnings.warn("Detected call of `lr_scheduler.step()` before `optimizer.step()`. "


Train Epoch: 2 [0/495 (0%)]	Loss: 1.217838	Accuracy of batch: 289.408000
Train Epoch: 2 [32/495 (6%)]	Loss: 1.218535	Accuracy of batch: 301.696000
Train Epoch: 2 [64/495 (12%)]	Loss: 1.217373	Accuracy of batch: 290.688000
Train Epoch: 2 [96/495 (19%)]	Loss: 1.218070	Accuracy of batch: 486.528000
Train Epoch: 2 [128/495 (25%)]	Loss: 1.217838	Accuracy of batch: 470.016000
